In [ ]:
## Cargo las librerias
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
 

##Cargo las librerias de omdb
import omdb
import requests

##Cargo librerias de scraping web
import re 
from bs4 import BeautifulSoup
import requests

##Para archivos y directorios
import io
import os
import chardet

##Librerias de tiempo
import time
from datetime import datetime
import time

##Libreria del zip
from zipfile import ZipFile

#Con esto podemos echar un vistazo a código HTML directamente en la Notebook
from IPython.display import HTML, display

##Librerías utilizadas para el muestreo de género de las peliculas
from collections import Counter
from random import uniform
from random import random
from bisect import bisect

###Librerias para el conteno de palabras (subtitulos)
from sklearn.feature_extraction.text import CountVectorizer
import nltk as nl
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import string as st
from textblob import TextBlob

##Separar training y test
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn import preprocessing
##Metricas
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, accuracy_score

##Modelos
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score


##Graficos
% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
################################################################################
##### Trabajo con los subtitulos descargados
################################################################################
def limpieza(fuente = './Subtitulos/SRT/', purgenewlines = False, lowercase = False):
    """
    Remueve los tiempos y tambien caracteres extranios. 
    """
    src = fuente
    hora_ini = datetime.now()
    
    ################# Limpieza #################
    timestamp = re.compile('^\d+\n?.*\n?', re.MULTILINE)
        #encuentra los numeros de indices y las duraciones
    advertise = re.compile ('Advertise your product or brand here\ncontact\w?')
    brackets = re.compile('\[[^]]*\]\n?|\([^)]*\)\n?|<[^>]*>\n?|\{[^}]*\}\n?')
        #encuentra corchetes y parentesis y lo que puede tener adentro, por ejemplo sonidos
    opensubs = re.compile('.*subtitles.*\n?|.*subs.*\n?', re.IGNORECASE)
        #encuentra las referencias a opensubtitles
    urls = re.compile('www.*\s\n?|[^\s]*\. ?com\n?')
        #encuentra paginas webs
    r = re.compile('\r')
        #encuentra returns
    puntuaciones = re.compile("[^\w\s']")
        #encuentra puntuaciones
    n = re.compile('\n')
        #encuentra nuevas lineas
    
    db_sub = pd.DataFrame()
    
    indice = 0
    total_archivos = len(os.listdir(src))
    
    for filename in os.listdir(src):
        try:
            if filename[-3:] == "srt":

                with open(os.path.join(src, filename), 'rb') as f:
                    result = chardet.detect(f.read())  # se puede usar readline si el archivo es muy largo

                texto = io.open(os.path.join(src, filename), mode="r", encoding=result['encoding'])
                texto = texto.read()
                
                if indice%50 == 0:
                    print ('Limpiando', filename)
                    print ('Total limpiados', indice+1, 'Faltan', round(((1-(indice+1)/total_archivos)*100),2),'%','\n')
                    print('Tiempo de ejecucion: {}'.format(datetime.now() - hora_ini))
                    
                texto = timestamp.sub('', texto)
                texto = advertise.sub('', texto)
                texto = brackets.sub('', texto)
                texto = opensubs.sub('', texto)
                texto = urls.sub('', texto)
                texto = r.sub('', texto)
                texto = puntuaciones.sub(' ', texto)
                if purgenewlines: 
                    texto = n.sub(' ', texto)
                if lowercase: 
                    texto = texto.lower()
                
                salida = {'imdb_id': filename[:-4], 'subtitulo': texto}
                salida = pd.DataFrame(salida, index= {indice})
                db_sub = pd.concat([db_sub, salida])
                indice += 1
        
        except Exception as e:
            print("Hubo un error en el id ", filename)
            print(e, "\n")
            continue
    
    print ("LISTO - Limpiados =", indice+1, '(',round((((indice+1)/total_archivos)*100),2),'%',')')
    return db_sub



In [ ]:
##Ejecuto la funcion de limpieza de subtitulos
db_sub = limpieza()

In [ ]:
##Guardo el subt
db_sub.to_csv('Subtitulos_limpios.csv', sep=';', index = False)